In [27]:
import pandas as pd 
import datetime as dt
import numpy as np
import keras
import plotly.graph_objects as go

from keras.models import Sequential
import tensorflow as tf
from alphaRNN import *
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l1
from sklearn.metrics import mean_squared_error, r2_score
from plotly.subplots import make_subplots
import statsmodels.api as sm

In [28]:
data = pd.read_csv("hourlySummaries.csv")
data = data.set_index("hour")
data = data[data.columns[1:]]
data = data.loc[:"2022-01-06 00:00:00"]
# data = data.iloc[:1000,]

train_weight = 0.80
split = int(len(data)*train_weight)
df_train = data.iloc[:split].copy()
df_test = data.iloc[split:].copy()

In [29]:
X_train = df_train.iloc[:,list(range(1,32))]
X_test = df_test.iloc[:,list(range(1,32))]

y_train = df_train["aaveHourlyChange"]
y_test = df_test["aaveHourlyChange"]
# y_train = df_train["aaveDirectionOfChange"]
# y_test = df_test["aaveDirectionOfChange"]

In [30]:
x_train_reg = np.array(X_train)
x_test_reg = np.array(X_test)
y_train_reg = np.array(y_train)
y_test_reg = np.array(y_test)

In [31]:
x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], x_train_reg.shape[1], 1))
x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], x_test_reg.shape[1], 1))

In [32]:
adf, p, usedlag, nobs, cvs,aic=sm.tsa.stattools.adfuller(y_train)

In [33]:
# Null hypothesis: Non Stationarity exists in the series.
# Alternative Hypothesis: Stationarity exists in the series

print("Critical Value: ", adf)
print("p-value" , p)
print("T-values for adfuller: ", cvs)
# Reject the null -- stationarity exists

Critical Value:  -15.685268746159464
p-value 1.4560319268665192e-28
T-values for adfuller:  {'1%': -3.431201534294984, '5%': -2.8619163150009173, '10%': -2.566970307948449}


In [34]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_train.index, y=df_test.aaveHourlyChange,
                    mode='lines',
                    name='Aave Change'),
                    secondary_y=True)

In [35]:
print(adf,p, nobs, cvs)

-15.685268746159464 1.4560319268665192e-28 7682 {'1%': -3.431201534294984, '5%': -2.8619163150009173, '10%': -2.566970307948449}


Data is stationary 
-16.70 > -3.43 

# Alpha RNN

In [36]:
def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
  model = Sequential()
  model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
  model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [37]:
max_epoches=1000
batch_size=1000

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=100, min_delta=1e-7, restore_best_weights=True)

In [38]:
model = AlphaRNN_(10,0.0)
model.fit(x_train_reg,y_train_reg,epochs=max_epoches, batch_size=batch_size,callbacks=[es],shuffle=False)


Epoch 1/1000
7719/7719 [==============================] - 6s 785us/step - loss: 29.5023
Epoch 2/1000
7719/7719 [==============================] - 1s 84us/step - loss: 29.3223
Epoch 3/1000
7719/7719 [==============================] - 1s 81us/step - loss: 29.1789
Epoch 4/1000
7719/7719 [==============================] - 1s 79us/step - loss: 29.0674
Epoch 5/1000
7719/7719 [==============================] - 1s 81us/step - loss: 28.9823
Epoch 6/1000
7719/7719 [==============================] - 1s 80us/step - loss: 28.9189
Epoch 7/1000
7719/7719 [==============================] - 1s 80us/step - loss: 28.8732
Epoch 8/1000
7719/7719 [==============================] - 1s 80us/step - loss: 28.8412
Epoch 9/1000
7719/7719 [==============================] - 1s 83us/step - loss: 28.8200
Epoch 10/1000
7719/7719 [==============================] - 1s 88us/step - loss: 28.8066
Epoch 11/1000
7719/7719 [==============================] - 1s 82us/step - loss: 28.7988
Epoch 12/1000
7719/7719 [===============

In [39]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
alpha_rnn_2 (AlphaRNN)       (None, 10)                121       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 132
Trainable params: 132
Non-trainable params: 0
_________________________________________________________________


In [40]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("Train MSE: ", MSE_train)
print("Train r2: ", r2_train)

y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)
r2_test = r2_score(y_test, y_pred, multioutput='variance_weighted')

print("Test MSE: ", MSE_test)
print("Test r2: ", r2_test)

1 - (1-r2_test)*(len(y_test_reg)-1)/(len(y_test_reg)-x_test_reg.shape[1]-1)

7719/7719 [==============================] - 2s 237us/step
Train MSE:  28.49690816814622
Train r2:  0.010308456280994527
1930/1930 [==============================] - 0s 108us/step
Test MSE:  7.898145331959788
Test r2:  -0.008351403825508141


-0.02482078924099329

In [48]:
pred_prices = []
for i in range(1,len(df_test)):
    price = df_test.aavePriceUSD[i-1] + y_pred[i]
    pred_prices.append(price[0])

In [50]:
pred_prices

[308.55298,
 305.73505,
 304.80554,
 305.28595,
 304.2442,
 297.27582,
 297.58612,
 301.90417,
 303.94092,
 306.41617,
 307.18408,
 307.7187,
 305.9414,
 303.3999,
 304.00586,
 303.93054,
 304.21176,
 301.43668,
 299.29745,
 295.9549,
 296.34103,
 294.8531,
 295.5621,
 297.77002,
 295.50568,
 294.68607,
 294.40887,
 293.30365,
 292.9346,
 295.8592,
 295.12918,
 294.32428,
 294.6839,
 294.7043,
 295.54916,
 296.83115,
 298.73584,
 298.93054,
 299.4528,
 298.53372,
 297.26157,
 297.2441,
 297.4827,
 298.25314,
 299.18887,
 300.53516,
 300.683,
 298.58588,
 299.51794,
 300.43417,
 302.07562,
 300.7433,
 301.45035,
 299.93176,
 300.3456,
 302.15698,
 304.0916,
 303.68832,
 301.89832,
 299.8401,
 298.46802,
 297.88858,
 297.74643,
 298.00833,
 298.77655,
 299.76987,
 299.35916,
 299.81137,
 299.82025,
 304.24173,
 308.3665,
 309.82767,
 311.5968,
 313.8161,
 313.4689,
 315.11658,
 315.92697,
 313.7814,
 316.2555,
 315.7982,
 316.5152,
 319.42334,
 324.69662,
 320.97882,
 321.9591,
 323.183,

In [51]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=df_test.index, y=pred_prices,
                    mode='lines',
                    name='Aave Predicted'),
                    secondary_y=False)
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.aavePriceUSD,
                    mode='lines',
                    name='Aave Actual'),
                    secondary_y=True)

# Using changes in price to predict numBorrows

In [ ]:
# X_train = df_train["aaveHourlyChange"]
# X_test = df_test["aaveHourlyChange"]

# y_train = df_train[["numBorrows"]]
# y_test = df_test[["numBorrows"]]

In [ ]:
# x_train_reg = np.array(X_train)
# x_test_reg = np.array(X_test)
# y_train_reg = np.array(y_train)
# y_test_reg = np.array(y_test)

In [ ]:
# x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], 1, 1))
# x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], 1, 1))

In [ ]:
# def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
#   model = Sequential()
#   model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
#   model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
#   model.compile(loss='mean_squared_error', optimizer='adam')
#   return model

In [ ]:
# model = AlphaRNN_(10,0.0)
# model.fit(x_train_reg,y_train_reg,epochs=max_epoches, batch_size=batch_size,callbacks=[es],shuffle=False)

In [ ]:
# model.summary()

In [ ]:
# y_pred_train = model.predict(x_train_reg, verbose = 1)
# MSE_train= mean_squared_error(y_train,y_pred_train)
# r2_train = r2_score(y_train, y_pred_train)

# print("Train MSE: ", MSE_train)
# print("Train r2: ", r2_train)

# y_pred = model.predict(x_test_reg, verbose = 1)
# MSE_test= mean_squared_error(y_test,y_pred)
# r2_test = r2_score(y_test, y_pred)

# print("Test MSE: ", MSE_test)
# print("Test r2: ", r2_test)